In [1]:
import json
import os
from pandas import DataFrame as df
import pandas as pd


# read files

In [3]:
IMPORT_PATH = "../data_enflate_cleaned"
filenames = os.listdir(IMPORT_PATH)
filenames = [file for file in filenames if file.endswith(".json")]
filename_list = [IMPORT_PATH + "/" + filename for filename in filenames]
devices_data = {}
for file in filename_list:
    device_data = pd.read_json(file)
    #print(device_data.head())
    devices_data[file] = device_data

In [12]:
devices_data_working = devices_data.copy()
devices_data_small = devices_data.copy()
#print(devices_data_working.keys())
print(f"found {len(devices_data_working.keys())} devices")

headers = ["Device", "Timestamp", "L1_min_current", "L1_max_current", "relay_state"]
for device in devices_data_working.values():
    headers = device.columns
    # delete all colums not in headers
    for column in device.columns:
        if column not in headers:
            device.drop(column, axis=1, inplace=True)


found 52 devices


# Calculate KPIs

In [13]:
devices_kpis = {}

# for device_name, device_data in devices_data_working.items():
#     device_kpis = {}
#     device_kpis["switching_count"] = calculate_switching_count(device_data)
#     devices_kpis[device_name] = device_kpis

# device_data = devices_data_small[filename_list[0]]
# def calculate_switching_count(device_data) -> int:
#     """loop over df and count the number of times the L1_min_voltage differs from L1_max_voltage"""
#     switching_count = 0
#     for index, row in device_data.iterrows():
#         # diff smaler 10 volts
#         if abs(row["L1_min_voltage"] - row["L1_max_voltage"]) > 10000:
#             switching_count += 1
#     return switching_count
# print (calculate_switching_count(device_data))


1
